In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from zoneinfo import ZoneInfo
import plotly.io as pio

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("googleplaystore.csv")

# -------------------------
# CLEAN INSTALLS
# -------------------------
df["Installs"] = (
    df["Installs"].astype(str)
    .str.replace("+","", regex=False)
    .str.replace(",","", regex=False)
)

df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# -------------------------
# FILTER — Category should NOT start with A,C,G,S
# -------------------------
df = df[~df["Category"].str.startswith(("A","C","G","S"), na=False)]

# -------------------------
# TOP 5 CATEGORIES BY INSTALLS
# -------------------------
cat_installs = (
    df.groupby("Category")["Installs"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .reset_index()
)

# -------------------------
# HIGHLIGHT > 1 MILLION
# -------------------------
cat_installs["Highlight"] = cat_installs["Installs"] > 1_000_000

# -------------------------
# TIME CHECK — 6PM–8PM IST
# -------------------------
now_ist = datetime.now(ZoneInfo("Asia/Kolkata"))

if 18 <= now_ist.hour < 20:

    # Assign sample ISO codes dynamically (must match row count)
    iso_codes = ["USA", "IND", "BRA", "GBR", "CAN"][:len(cat_installs)]

    fig = px.choropleth(
        cat_installs,
        locations=iso_codes,
        locationmode="ISO-3",
        color="Installs",
        hover_name="Category",
        hover_data=["Installs", "Highlight"],
        title="Global Installs by Top 5 Categories"
    )

    pio.renderers.default = "browser"
    fig.show()

else:
    print("Map hidden — visible only between 6 PM and 8 PM IST")


Map hidden — visible only between 6 PM and 8 PM IST
